# Cleaning and Merging Suicide and Population Datasets

### Data Sources
- Suicide dataset: https://vizhub.healthdata.org/gbd-results/
- Population dataset: https://ghdx.healthdata.org/record/ihme-data/gbd-2019-population-estimates-1950-2019
- Location ID dataset: https://ghdx.healthdata.org/sites/default/files/ihme_query_tool/IHME_GBD_2019_CODEBOOK.zip
- WHO World Standard Population (page 12): https://cdn.who.int/media/docs/default-source/gho-documents/global-health-estimates/gpe_discussion_paper_series_paper31_2001_age_standardization_rates.pdf

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import country_converter as coco

## Suicide Dataset
### File Location:
First, let us import our suicides dataset.

In [2]:
path_suicides = r'raw data/IHME-GBD_2019_DATA-5c898f39.csv'
df_suicides_csv = pd.read_csv(path_suicides)
df_suicides_csv

,measure,location,sex,age,cause,metric,year,val,upper,lower
0,Deaths,Malaysia,Male,10-14 years,Self-harm,Number,2006,3.305274,9.015942,1.745501
1,Deaths,Malaysia,Female,10-14 years,Self-harm,Number,2006,1.040353,1.577221,0.657604
2,Deaths,Malaysia,Male,15-19 years,Self-harm,Number,2006,69.972807,137.531684,46.508597
3,Deaths,Malaysia,Female,15-19 years,Self-harm,Number,2006,18.619116,25.274600,13.020568
4,Deaths,Malaysia,Male,20-24 years,Self-harm,Number,2006,101.780071,179.316911,72.770759
...,...,...,...,...,...,...,...,...,...,...
91387,Deaths,Republic of Cabo Verde,Female,75-79 years,Self-harm,Number,2019,0.696599,1.081157,0.431770
91388,Deaths,Republic of Cabo Verde,Male,80-84,Self-harm,Number,2019,2.158448,3.668295,1.367896
91389,Deaths,Republic of Cabo Verde,Female,80-84,Self-harm,Number,2019,0.972983,1.551506,0.595383
91390,Deaths,Republic of Cabo Verde,Male,85+ years,Self-harm,Number,2019,1.962957,3.505776,1.365964


### Column Cleanup:
We only need the following columns: `location`, `sex`, `age`, `year`, `val`
- Keep only the necessary columns
- Renaming columns for readability
- Making all values lower case in column `sex` for later merge with population dataset, because it is case-sensitive

In [3]:
df_suicides = (df_suicides_csv[['location', 
                                'sex',
                                'age',
                                'year', 
                                'val']]
              .rename(columns = {'location': 'country',
                                 'val': 'suicides'})
              )

In [4]:
df_suicides['sex'] = [sex.lower() for sex in df_suicides['sex']]

### Standardizing Age Values
The age values provided are not standardized, so let's standardize them.

In [5]:
df_suicides['age'].unique()

array(['10-14 years', '15-19 years', '20-24 years', '25-29 years',
       '30-34 years', '35-39 years', '40-44 years', '45-49 years',
       '50-54 years', '55-59 years', '60-64 years', '65-69 years',
       '70-74 years', '75-79 years', '80-84', '85+ years'], dtype=object)

In [6]:
df_suicides['age'] = df_suicides['age'].replace(to_replace = '80-84', value = '80-84 years')

In [7]:
df_suicides

,country,sex,age,year,suicides
0,Malaysia,male,10-14 years,2006,3.305274
1,Malaysia,female,10-14 years,2006,1.040353
2,Malaysia,male,15-19 years,2006,69.972807
3,Malaysia,female,15-19 years,2006,18.619116
4,Malaysia,male,20-24 years,2006,101.780071
...,...,...,...,...,...
91387,Republic of Cabo Verde,female,75-79 years,2019,0.696599
91388,Republic of Cabo Verde,male,80-84 years,2019,2.158448
91389,Republic of Cabo Verde,female,80-84 years,2019,0.972983
91390,Republic of Cabo Verde,male,85+ years,2019,1.962957


### Standardizing Country Names:
We need to standardize the names of countries for later merging with populations dataset.
- Standardize all country names using `country_converter` package (https://pypi.org/project/country-converter/)

In [8]:
countries_s = df_suicides['country'].unique()
standard_country_names_s = coco.convert(names = countries_s, 
                                      to = 'name_short')

In [9]:
country_fix_dict_s = dict(zip(countries_s, standard_country_names_s))
df_suicides['country'] = df_suicides['country'].replace(country_fix_dict_s)

In [10]:
df_suicides

,country,sex,age,year,suicides
0,Malaysia,male,10-14 years,2006,3.305274
1,Malaysia,female,10-14 years,2006,1.040353
2,Malaysia,male,15-19 years,2006,69.972807
3,Malaysia,female,15-19 years,2006,18.619116
4,Malaysia,male,20-24 years,2006,101.780071
...,...,...,...,...,...
91387,Cabo Verde,female,75-79 years,2019,0.696599
91388,Cabo Verde,male,80-84 years,2019,2.158448
91389,Cabo Verde,female,80-84 years,2019,0.972983
91390,Cabo Verde,male,85+ years,2019,1.962957


### Removing Suicide Count Estimates Below 1
- Suicide count in the dataset is an estimate, and many values are estimated to be much lower than 1 suicide. These will be replaced by values of 0 to avoid inaccuracies in calculating suicide rates later.

In [11]:
cutoff = 1
df_suicides['suicides'] = np.where(df_suicides['suicides'] <= cutoff, 0, df_suicides['suicides'])

## Population Dataset

### Reading and Combining .csv Files:
Every year of population data is kept its own .csv file.
- Read and concatenate population .csv files

In [12]:
path_populations = r'raw data/population data'
all_files = glob.glob(os.path.join(path_populations, "*.csv"))

df_from_each_file = (pd.read_csv(f) for f in all_files)
df_populations_csv = pd.concat(df_from_each_file, ignore_index=True)
df_populations_csv

,location_id,location_name,sex_id,sex_name,age_group_id,age_group_name,year_id,measure_id,measure_name,metric_id,metric_name,val,upper,lower
0,1,Global,1,male,1,Under 5,1990,44,Population,1,Number,3.253906e+08,3.323687e+08,3.186139e+08
1,1,Global,1,male,2,Early Neonatal,1990,44,Population,1,Number,1.349236e+06,1.377880e+06,1.321656e+06
2,1,Global,1,male,3,Late Neonatal,1990,44,Population,1,Number,3.977538e+06,4.061891e+06,3.896573e+06
3,1,Global,1,male,4,Post Neonatal,1990,44,Population,1,Number,6.248355e+07,6.380593e+07,6.121612e+07
4,1,Global,1,male,5,1 to 4,1990,44,Population,1,Number,2.575802e+08,2.630934e+08,2.521675e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855245,44638,North America,2,female,164,Birth,2019,44,Population,1,Number,2.050261e+06,2.193948e+06,1.919386e+06
2855246,44638,North America,3,both,164,Birth,2019,44,Population,1,Number,4.200444e+06,4.494820e+06,3.932317e+06
2855247,44642,Nordic Region,1,male,164,Birth,2019,44,Population,1,Number,1.498598e+05,1.642737e+05,1.369929e+05
2855248,44642,Nordic Region,2,female,164,Birth,2019,44,Population,1,Number,1.419441e+05,1.556075e+05,1.297482e+05


### Column Cleaning:
We only need the following columns: `location_id`, `location_name`, `sex_name`, `age_group_name`, `year_id`, `val`
- Keep only the necessary columns
- Rename these columns for readability

In [13]:
df_populations = (df_populations_csv[['location_id',
                                      'location_name', 
                                      'sex_name', 
                                      'age_group_name', 
                                      'year_id',
                                      'val']]
          .rename(columns = {'location_name': 'country', 
                             'sex_name': 'sex',
                             'age_group_name': 'age',
                             'year_id': 'year', 
                             'val': 'population'})
     )

### Row Cleaning:
The dataset includes non-gender-specific data, which we do not need.
- Removing unnecessary rows

In [14]:
df_populations = df_populations[df_populations['sex'].isin(['male', 'female'])]

In [15]:
df_populations

,location_id,country,sex,age,year,population
0,1,Global,male,Under 5,1990,3.253906e+08
1,1,Global,male,Early Neonatal,1990,1.349236e+06
2,1,Global,male,Late Neonatal,1990,3.977538e+06
3,1,Global,male,Post Neonatal,1990,6.248355e+07
4,1,Global,male,1 to 4,1990,2.575802e+08
...,...,...,...,...,...,...
2855242,44591,World Bank Income Levels,female,Birth,2019,6.526385e+07
2855244,44638,North America,male,Birth,2019,2.150183e+06
2855245,44638,North America,female,Birth,2019,2.050261e+06
2855247,44642,Nordic Region,male,Birth,2019,1.498598e+05


### Filter Down by Age Column:
- Filter down dataset by `age` values we want (`10-14 years`, `15-19 years`, etc.)
- Rename the `age` values for consistency and to match the written format of the suicides dataset, i.e.: `25-29 years`

In [16]:
df_populations['age'].unique()

array(['Under 5', 'Early Neonatal', 'Late Neonatal', 'Post Neonatal',
       '1 to 4', '5 to 9', '10 to 14', '15 to 19', '20 to 24', '25 to 29',
       '30 to 34', '35 to 39', '40 to 44', '45 to 49', '50 to 54',
       '55 to 59', '60 to 64', '65 to 69', '70 to 74', '75 to 79',
       '80 plus', 'All Ages', '5-14 years', '15-49 years', '50-69 years',
       '70+ years', '<1 year', '80 to 84', '85 to 89', '90 to 94',
       '0 to 14', 'Neonatal', '25 plus', '<20 years', '10 to 24',
       '10 to 19', 'Birth', '95 plus', '5 to 19', '15 to 39', '25 to 49',
       '65 to 74', '75 to 84', '20 plus', '55 plus', '75 plus', '85 plus'],
      dtype=object)

In [17]:
df_populations = df_populations[df_populations['age'].isin(['10 to 14',
                                                            '15 to 19',
                                                            '20 to 24',
                                                            '25 to 29',
                                                            '30 to 34',
                                                            '35 to 39',
                                                            '40 to 44',
                                                            '45 to 49',
                                                            '50 to 54',
                                                            '55 to 59',
                                                            '60 to 64',
                                                            '65 to 69',
                                                            '70 to 74',
                                                            '75 to 79',
                                                            '80 to 84',
                                                            '85 plus'])]

In [18]:
df_populations['age'] = (df_populations['age'].replace(to_replace = '85 plus', value = '85+')
                                              .replace(' to ',  '-', regex=True) + ' years'
                        )

C:\Users\croww\AppData\Local\Temp\ipykernel_26196\543736996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_populations['age'] = (df_populations['age'].replace(to_replace = '85 plus', value = '85+')


### Filtering Out Non-Country `location_id`]
We have a number of non-country locations in our populations dataset:

In [19]:
df_populations['country'].unique()[:10]

array(['Global', 'Southeast Asia, East Asia, and Oceania', 'East Asia',
       'China', "Democratic People's Republic of Korea",
       'Taiwan (Province of China)', 'Southeast Asia', 'Cambodia',
       'Indonesia', "Lao People's Democratic Republic"], dtype=object)

The [Location ID Codebook dataset](https://ghdx.healthdata.org/sites/default/files/ihme_query_tool/IHME_GBD_2019_CODEBOOK.zip) taken from the IHME website can let us filter it down.
- Read Location ID dataset
- Sort by `Level = 3`, which is country-level
- Filter out non-country locations from populations dataset

In [20]:
path_locations = r'IHME_GBD_2019_CODEBOOK/IHME_GBD_2019_GBD_LOCATION_HIERARCHY_Y2022M06D29.XLSX'
df_locations_csv = pd.read_excel(path_locations)
df_locations_csv

,Location Set Version ID,Location ID,Location Nam,Parent ID,Level,Sort Order
0,1073,1,Global,1,0,1
1,1073,44637,Low SDI,1,1,2
2,1073,44636,Low-middle SDI,1,1,3
3,1073,44639,Middle SDI,1,1,4
4,1073,44634,High-middle SDI,1,1,5
...,...,...,...,...,...,...
705,1073,214,Nigeria,199,3,706
706,1073,215,Sao Tome and Principe,199,3,707
707,1073,216,Senegal,199,3,708
708,1073,217,Sierra Leone,199,3,709


In [21]:
df_locations = df_locations_csv[['Location ID',
                                 'Location Nam',
                                 'Level']]

In [22]:
df_locations = df_locations[df_locations['Level'] == 3]
location_ids = df_locations['Location ID']

In [23]:
df_populations = df_populations[df_populations['location_id'].isin(location_ids)]
df_populations

,location_id,country,sex,age,year,population
348,6,China,male,10-14 years,1990,5.305136e+07
349,6,China,male,15-19 years,1990,6.516001e+07
350,6,China,male,20-24 years,1990,6.768973e+07
351,6,China,male,25-29 years,1990,5.655363e+07
352,6,China,male,30-34 years,1990,4.615589e+07
...,...,...,...,...,...,...
2843320,422,United States Virgin Islands,female,85+ years,2019,6.852529e+02
2843400,435,South Sudan,male,85+ years,2019,8.019785e+03
2843401,435,South Sudan,female,85+ years,2019,8.041591e+03
2843481,522,Sudan,male,85+ years,2019,4.880277e+04


### Standardizing Country Names:
- Standardize all country names using `country_converter` package (https://pypi.org/project/country-converter/)

In [24]:
countries_p = df_populations['country'].unique()
standard_country_names_p = coco.convert(names = countries_p, 
                                      to = 'name_short')

In [25]:
country_fix_dict_p = dict(zip(countries_p, standard_country_names_p))
df_populations['country'] = df_populations['country'].replace(country_fix_dict_p)

## Merging Datasets
Now we merge the suicides and populations datasets on `country`, `sex`, `age`, `year`.

In [26]:
df_merged = df_populations.merge(df_suicides,
                                 how = 'inner',
                                 on = ['country', 'sex', 'age', 'year'])
df_merged

,location_id,country,sex,age,year,population,suicides
0,6,China,male,10-14 years,2006,4.991414e+07,715.356058
1,6,China,male,15-19 years,2006,6.744827e+07,3437.717644
2,6,China,male,20-24 years,2006,5.543059e+07,4213.977394
3,6,China,male,25-29 years,2006,4.973199e+07,3754.931093
4,6,China,male,30-34 years,2006,5.997560e+07,5242.494777
...,...,...,...,...,...,...,...
91387,422,United States Virgin Islands,female,85+ years,2019,6.852529e+02,0.000000
91388,435,South Sudan,male,85+ years,2019,8.019785e+03,8.093728
91389,435,South Sudan,female,85+ years,2019,8.041591e+03,2.468230
91390,522,Sudan,male,85+ years,2019,4.880277e+04,7.727045


## Add World Standard Population Distribution Values
As stated in the section on age-standardization in the main article, we need population weightings for each age group to perform age-standardization.
- Add the population weightings for each age group for age-standardization later
- Visit [here](https://www3.paho.org/hq/dmdocuments/2012/PAHO-Age-standardized-mortality-rates-2009.pdf) for more information on age-standardization

In [27]:
ages10 = 8.60
ages15 = 8.47
ages20 = 8.22
ages25 = 7.93
ages30 = 7.61
ages35 = 7.15
ages40 = 6.59
ages45 = 6.04
ages50 = 5.37
ages55 = 4.55
ages60 = 3.72
ages65 = 2.96
ages70 = 2.21
ages75 = 1.52
ages80 = 0.91
ages85 = 0.44 + 0.15 + 0.04 + 0.005

In [28]:
raw = [ages10,
       ages15,
       ages20,
       ages25,
       ages30,
       ages35,
       ages40,
       ages45,
       ages50,
       ages55,
       ages60,
       ages65,
       ages70,
       ages75,
       ages80,
       ages85,]
norm = [float(i)/sum(raw) for i in raw]
norm

[0.10426138085712557,
 0.10268533672788996,
 0.09965448263320607,
 0.09613869188337276,
 0.0922591986421774,
 0.08668242710795905,
 0.07989331393586716,
 0.07322543492756262,
 0.06510274595380981,
 0.05516154452324667,
 0.04509910892889618,
 0.035885312481057176,
 0.026792750197005524,
 0.018427592895678007,
 0.011032308904649334,
 0.007698369400497063]

In [29]:
age_groups = df_populations['age'].unique()
age_groups

array(['10-14 years', '15-19 years', '20-24 years', '25-29 years',
       '30-34 years', '35-39 years', '40-44 years', '45-49 years',
       '50-54 years', '55-59 years', '60-64 years', '65-69 years',
       '70-74 years', '75-79 years', '80-84 years', '85+ years'],
      dtype=object)

In [30]:
weights = dict(zip(age_groups, norm))
weights

{'10-14 years': 0.10426138085712557,
 '15-19 years': 0.10268533672788996,
 '20-24 years': 0.09965448263320607,
 '25-29 years': 0.09613869188337276,
 '30-34 years': 0.0922591986421774,
 '35-39 years': 0.08668242710795905,
 '40-44 years': 0.07989331393586716,
 '45-49 years': 0.07322543492756262,
 '50-54 years': 0.06510274595380981,
 '55-59 years': 0.05516154452324667,
 '60-64 years': 0.04509910892889618,
 '65-69 years': 0.035885312481057176,
 '70-74 years': 0.026792750197005524,
 '75-79 years': 0.018427592895678007,
 '80-84 years': 0.011032308904649334,
 '85+ years': 0.007698369400497063}

In [31]:
df_merged['weights'] = df_merged['age'].map(weights)

In [32]:
df_merged

,location_id,country,sex,age,year,population,suicides,weights
0,6,China,male,10-14 years,2006,4.991414e+07,715.356058,0.104261
1,6,China,male,15-19 years,2006,6.744827e+07,3437.717644,0.102685
2,6,China,male,20-24 years,2006,5.543059e+07,4213.977394,0.099654
3,6,China,male,25-29 years,2006,4.973199e+07,3754.931093,0.096139
4,6,China,male,30-34 years,2006,5.997560e+07,5242.494777,0.092259
...,...,...,...,...,...,...,...,...
91387,422,United States Virgin Islands,female,85+ years,2019,6.852529e+02,0.000000,0.007698
91388,435,South Sudan,male,85+ years,2019,8.019785e+03,8.093728,0.007698
91389,435,South Sudan,female,85+ years,2019,8.041591e+03,2.468230,0.007698
91390,522,Sudan,male,85+ years,2019,4.880277e+04,7.727045,0.007698


## Exporting to .csv File:

In [33]:
path_export = r'cleaned data/Suicides_Population_Merged.csv'
df_merged.to_csv(path_export)